In [1]:
import pandas as pd
import csv
import os 
import datetime as dt
import teradatasql as td
from selenium import webdriver
from selenium.webdriver.common.by import By
import datetime
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from pathlib import Path

In [2]:
print(os.listdir('C:/Users/Chweber/Downloads'))
print(os.getcwd())

['#CM. Smart Player Outreach (NBA) - Las Vegas 1-4-2025 11-24-05 AM.xlsx', 'desktop.ini', '~$All Leads - Measurement 10-24-2024 3-05-41 PM.xlsx', '~$High Value New Gamer - New to MGM.pptx']
C:\Users\Chweber\Host NBA CRM Stuff\Host NBA Pipeline


<h1> Host NBA CRM Data Pipeline </h1>

<h3>Data Pulling</h3>

In [7]:
host_dynamics_urls = ['https://mgmhost.crm.dynamics.com/main.aspx?appid=df81b847-7ed4-4d00-bbc4-01b31d607d2a&pagetype=entitylist&etn=task&viewid=5a944623-fe03-ee11-8f6e-6045bd006149&viewType=4230&lid=1722024607660'
                      , 'https://mgmhost.crm.dynamics.com/main.aspx?appid=df81b847-7ed4-4d00-bbc4-01b31d607d2a&pagetype=entitylist&etn=task&viewid=356ac88c-fc03-ee11-8f6e-6045bd006793&viewType=4230'
                      , 'https://mgmhost.crm.dynamics.com/main.aspx?appid=df81b847-7ed4-4d00-bbc4-01b31d607d2a&pagetype=entitylist&etn=task&viewid=feb7d7bc-fc03-ee11-8f6e-6045bd006793&viewType=4230']

with open('asdf.txt','r') as file2:
    pw = file2.read()
    
# download_dir = 'C:/Users/Chweber/Host NBA CRM Stuff/CRM Files'

chrome_options = webdriver.ChromeOptions()

driver = webdriver.Chrome(options=chrome_options)
driver.get(host_dynamics_urls[0])
# print(host_dynamics_urls[0])
input_element = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, 'input.form-control.ltr_override.input.ext-input.text-box.ext-text-box'))
)
# text_entry = driver.find_element(By.CSS_SELECTOR, 'input.form-control.ltr_override.input.ext-input.text-box.ext-text-box')
input_element.send_keys('cweber@mgmresorts.com')
input_element.send_keys(Keys.ENTER)

input_element = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.ID, 'input28'))
)
input_element.send_keys('604591')
input_element.send_keys(Keys.ENTER)

input_element = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.ID, 'input54'))
)
input_element.send_keys(pw)
input_element.send_keys(Keys.ENTER)

files = os.listdir('C:/Users/Chweber/Downloads')
dwnld_length = len(files)
set = dwnld_length

n=1
try:
    while dwnld_length != set+3:
        timeout = 1000
    
        # waits until button loads to click
        button = WebDriverWait(driver, timeout).until(
            EC.presence_of_element_located((By.ID, 'activitypointer|NoRelationship|HomePageGrid|Mscrm.HomepageGrid.activitypointer.ExportToExcel.Menu$button0'))
        )
        button.click()
        print('button clicked successfully')
    
        # gets length of the current downloads directory
        files = os.listdir('C:/Users/Chweber/Downloads')
        set2 = len(files)
    
        # until the file has finished downloading, keep this loop going
        while dwnld_length != set2+1:
            print('bedtime')
            time.sleep(60)
            files = os.listdir('C:/Users/Chweber/Downloads')
            dwnld_length = len(files)
    
        # get next download, caps it at three for each of the host nba markets
        # while n < 3:
        driver.get(host_dynamics_urls[n])
        n+=1
        time.sleep(20)
except:
    print('All files downloaded')


button clicked successfully
bedtime
bedtime
bedtime
button clicked successfully
bedtime
bedtime
button clicked successfully
bedtime
All files downloaded


In [13]:
# gets files from the directory
files = os.listdir('C:/Users/Chweber/Downloads')

# specifically the three nba files just downloaded
files = files[:3]

for i in files:
    # reads each excel file
    excel_file = 'C:/Users/Chweber/Downloads/'+i
    # puts file into a dataframe
    df = pd.read_excel(excel_file, engine='openpyxl')
    # df.to_csv('Host NBA Pipeline/CRM Files/'+i[:-5]+'.csv')
    df.to_csv('CRM Files/'+i[:-5]+'.csv')

C:\Anaconda\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Anaconda\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Anaconda\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


<h3>Data Cleansing/Validation</h3>

In [25]:
# file_path = os.listdir('Host NBA Pipeline/CRM Files')
file_path = os.listdir('CRM Files')
# applies data transformations necessary for SQL queries
for i in file_path[:3]:
    # df = pd.read_csv('Host NBA Pipeline/CRM Files/'+i)
    df = pd.read_csv('CRM Files/'+i)
    # removes unwanted columns
    df = df.drop(columns=['(Do Not Modify) Task','(Do Not Modify) Row Checksum','(Do Not Modify) Modified On','Unnamed: 0', 'Task Completion Comments'])

    # creates Dateloaded column, formats as appropriate for a timestamp value
    df['Dateloaded'] = pd.Timestamp.now().floor('s')
    df['Dateloaded'] = df['Dateloaded'].astype(str)
    df['Dateloaded'] = pd.to_datetime(df['Dateloaded'].str.replace(r'[^\d\-: ]', '', regex=True), errors='coerce').dt.normalize()
    df['Dateloaded'] = df['Dateloaded'].dt.strftime('%m-%d-%Y %H:%M:%S')

    # converts the columns representing dates to the datetime format
    df['Created On'] = pd.to_datetime(df['Created On'].str.replace(r'[^\d\-: ]', '', regex=True), errors='coerce').dt.normalize()
    df['Due Date'] = pd.to_datetime(df['Due Date'].str.replace(r'[^\d\-: ]', '', regex=True), errors='coerce').dt.normalize()
    df['Actual End'] = pd.to_datetime(df['Actual End'].str.replace(r'[^\d\-: ]', '', regex=True), errors='coerce').dt.normalize()

    # removes all whitespace, necessary for SQL
    df = df.rename(columns={'Created On':'Created_On','Host ID (Owning User) (User)':'Host_ID','User Name (Owning User) (User)':'User_Name','Property (Owning User) (User)':'Property','MGM Rewards # (Regarding) (Contact)':'MGM_Rewards_ID','Due Date':'Due_Date','Actual End':'Actual_End','Activity Status':'Activity_Status','Channel Outcome':'Channel_Outcome','NBA ID':'NBA_ID','NBA Type':'NBA_Type'})
    
    # removes duplicate NBA tasks
    df = df.drop_duplicates()

    # fills all na with 0
    df = df.fillna(0)

    # ensures data types for each column are appropriate
    df['Created_On'] = df['Created_On'].astype(str)
    df['Owner'] = df['Owner'].astype(str)
    df['Host_ID'] = df['Host_ID'].astype(int)
    df['User_Name'] = df['User_Name'].astype(str)
    df['Property'] = df['Property'].astype(str)
    df['Subject'] = df['Subject'].astype(str)
    df['MGM_Rewards_ID'] = df['MGM_Rewards_ID'].astype(int)
    df['Regarding'] = df['Regarding'].astype(str)
    df['ADW'] = df['ADW'].astype(int)
    df['Due_Date'] = df['Due_Date'].astype(str)
    df['Actual_End'] = df['Actual_End'].astype(str)
    df['Activity_Status'] = df['Activity_Status'].astype(str)
    df['Channel'] = df['Channel'].astype(str)
    df['Channel_Outcome'] = df['Channel_Outcome'].astype(str)
    df['NBA_ID'] = df['NBA_ID'].astype(int)
    df['NBA_Type'] = df['NBA_Type'].astype(str)

    # uses list comprehension to add Dateloaded as the first column
    columns = ['Dateloaded'] + [col for col in df.columns if col != 'Dateloaded']
    
    # specifies order of columns
    df = df[columns]
        
    # creates prepped versions of the dataframes, tab as the delimiter which is necessary for teradata sql import
    # df.to_csv('Host NBA Pipeline/Prepped CRM Files/prepped_'+i[:-4]+'.txt', sep = '\t', index=False)
    df.to_csv('Prepped CRM Files/prepped_'+i[:-4]+'.txt', sep = '\t', index=False)

<h3>Teradata Driver Integration</h3>

In [35]:
# specifies names of the tables 
table_names = ['ae_smart_player_outreach_lv','ae_smart_player_outreach_pcomm','ae_smart_player_outreach_reg']

# grabs location of prepped files
prepped_files = os.listdir('Prepped CRM Files')

# grabs location of sql queries
queries = os.listdir('SQL Queries')

# purpose of the function is to create/refresh the tables
def table_creation(cur,n):
    print('Creating table...')
    try:
        cur.execute("""
                    drop table prd_mktg_opr_tb."""+table_names[n]
                    )
    except:
        print('table not found')
        
    cur.execute ("""
                create table prd_mktg_opr_tb."""+table_names[n]+""" (
                    Dateloaded TIMESTAMP,
                    Created_On varchar(255),
                    "Owner" varchar(255),
                    Host_ID integer,
                    User_Name varchar(255),
                    Property varchar(255),
                    Subject varchar(255),
                    MGM_Rewards_ID integer,
                    Regarding varchar(255),
                    ADW integer,
                    Due_Date varchar(255),
                    Actual_End varchar(255),
                    Activity_Status varchar(255),
                    Channel varchar(255),
                    Channel_Outcome varchar(255),
                    NBA_ID integer,
                    NBA_Type varchar(255)
                    );
                  """)
    print('Table creation successful for '+table_names[n])

# purpose of the function is to import the prepped files into the newly created tables
def import_to_teradata(df,cur):
    try:
        # necessary for data import, list out columns and then ? as the wildcard
        columns = ', '.join(df.columns)
        placeholders = ', '.join(['?' for i in range(len(df.columns)-1)])
        print(columns+'\n'+placeholders)
        
        print('Inserting values...')
        # sql statement
        sql = f"INSERT INTO prd_mktg_opr_tb.{table_names[n]} ({columns}) VALUES ({placeholders})"

        # rows
        data_to_insert = [tuple(row) for row in df.values]

        # directs cursor to import the data 
        cur.executemany(sql, data_to_insert)

    except Exception as e:
        print(f'An error occurred: {e}')
        try:
            cur.execute("SELECT {fn teradata_get_errors()} AS error_details")
            error_details = cursor.fetchone()
            print("Error Details:", error_details)
            
            cur.execute("SELECT {fn teradata_nativesql()} AS native_sql_error")
            native_sql_error = cursor.fetchone()
            print("Native SQL Error:", native_sql_error)

        except Exception as inner_e:
            print(f"An error occurred while retrieving error details: {inner_e}")
            
    finally:
        print("Data import successful")
    
def execute_query(cur,lines): 
    # breaks apart each query into its individual statements
    statements = lines.split(';')

    # removes whitespace
    statements = [i.strip() for i in statements]
    # statements = [i.replace('\n', '').replace('\t', '') for i in statements]

    # grab the sql statements which involve volatile table creation, select statements, etc.
    keywords = ["create", "CREATE", "sel", "SELECT",'insert','INSERT']
    
    # only grab the statements which have a match from the list above
    statements = [i for i in statements if any(keyword in i for keyword in keywords)]

    print('Executing query...')

    # iterate over the statements, executing them with the cursor
    for i in statements:
        print('\t'+i[:30])
        cur.execute(i)
    print('Query executed')

# grabs password
with open('asdf.txt','r') as file2:
    pw = file2.read()

# for each table which needs refreshing
for n in range(3):
    # script_dir = os.path.dirname(os.path.abspath(Path.cwd()))
    
    # getting the absolute path to your prepped file and queries file
    prepped_file_path = os.path.join('Prepped CRM Files/',prepped_files[n+1])
    queries_file_path = os.path.join('SQL Queries/',queries[n+1])
    
    # prepped_file_path = 'CRM Files/'+prepped_files[n+1]
    # reading each CSV file and queries file using the absolute path
    df = pd.read_csv(prepped_file_path,sep='\t')

    # connects to Teradata server, specifies host, logmech, and user to execute queries
    with td.connect(host='TDPROD',logmech='LDAP',user='chweber',password=pw) as con:
        with con.cursor () as cur: 
            # create table
            table_creation(cur,n)
            # import prepped data to these tables
            import_to_teradata(df,cur)
            # reads query files, passes it to the execute function
            with open(queries_file_path,'r') as file:
                lines = file.read()
            # execute queries, 40+ statements in each
            execute_query(cur,lines)

Creating table...
Table creation successful for ae_smart_player_outreach_lv
Dateloaded, Created_On, Owner, Host_ID, User_Name, Property, Subject, MGM_Rewards_ID, Regarding, ATT (Corp), ADW, Due_Date, Actual_End, Activity_Status, Channel, Channel_Outcome, NBA_ID, NBA_Type
?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?
Inserting values...
An error occurred: [Version 20.0.0.15] [Session 8317162] [Teradata SQL Driver] [Error 528] A failure occurred while executing rows 98299 through 99911 of a batch request. Details of the failure can be found in the error string that is accessible using the escape functions {fn teradata_nativesql}{fn teradata_get_errors}.
 at gosqldriver/teradatasql.formatError ErrorUtil.go:91
 at gosqldriver/teradatasql.(*teradataConnection).makeDriverErrorCode ErrorUtil.go:212
 at gosqldriver/teradatasql.(*TeradataRows).generateBatchErrors TeradataRows.go:929
 at gosqldriver/teradatasql.newTeradataRows TeradataRows.go:681
 at gosqldriver/teradatasql.(*teradataStateme

OperationalError: [Version 20.0.0.15] [Session 8317162] [Teradata Database] [Error 3776] Comment not terminated before end of request text.
 at gosqldriver/teradatasql.formatError ErrorUtil.go:91
 at gosqldriver/teradatasql.(*teradataConnection).formatDatabaseError ErrorUtil.go:251
 at gosqldriver/teradatasql.(*teradataConnection).makeChainedDatabaseError ErrorUtil.go:267
 at gosqldriver/teradatasql.(*teradataConnection).processErrorParcel TeradataConnection.go:715
 at gosqldriver/teradatasql.(*TeradataRows).processResponseBundle TeradataRows.go:2308
 at gosqldriver/teradatasql.(*TeradataRows).executeSQLRequest TeradataRows.go:874
 at gosqldriver/teradatasql.newTeradataRows TeradataRows.go:720
 at gosqldriver/teradatasql.(*teradataStatement).QueryContext TeradataStatement.go:122
 at gosqldriver/teradatasql.(*teradataConnection).QueryContext TeradataConnection.go:1225
 at database/sql.ctxDriverQuery ctxutil.go:48
 at database/sql.(*DB).queryDC.func1 sql.go:1776
 at database/sql.withLock

In [ ]:
print(prepped_files)

In [ ]:
for i in range(3):
    print(i+1)

In [ ]:
for n in range(3):
    print(n)

In [ ]:
os.getcwd()

In [ ]:
print(os.getcwd())

In [ ]:
with open("Host NBA Pipeline/CRM Files/prepped_#CM. Smart Player Outreach (NBA) - Las Vegas 11-14-2024 10-15-12 AM.txt", "r") as file:
    content = file.read()
    print(content)

In [ ]:
folder_path = "Host NBA Pipeline/CRM Files"
file_name = "prepped_#CM. Smart Player Outreach (NBA) - Las Vegas 11-14-2024 10-15-12 AM.txt"

file_path = os.path.join(folder_path, file_name)

if os.path.exists(file_path):
    print("File exists")
else:
    print("File does not exist")

In [ ]:
import os
import pandas as pd

# Define the relative path
relative_path = "CRM Files/prepped_#CM. Smart Player Outreach (NBA) - Las Vegas 11-14-2024 10-15-12 AM.txt"

# Ensure the path is correct and formatted properly
absolute_path = os.path.abspath(relative_path)
print(f"Attempting to read from: {absolute_path}")

# Check if file exists
if os.path.exists(absolute_path):
    print("File found!")
    try:
        # Read the CSV file
        df = pd.read_csv(absolute_path)
        print("File read successfully!")
        print(df.head())  # Optionally display the first few rows to confirm
    except Exception as e:
        print(f"Error reading the file: {e}")
else:
    print(f"File does not exist at: {absolute_path}")

In [ ]:
with open('test2.txt','r') as file:
    lines = file.read()

with open('asdf.txt','r') as file2:
    pw = file2.read()

statements = lines.split(';')
statements = [i.strip() for i in statements]
# statements = [i.replace('\n', '').replace('\t', '') for i in statements]

keywords = ["create", "CREATE", "sel", "SELECT"]
# Filter the list to keep only elements containing any of the keywords
statements = [i for i in statements if any(keyword in i for keyword in keywords)]

# print(statements)

with td.connect(host='TDPROD',logmech='LDAP',user='chweber',password=pw) as con:
    with con.cursor () as cur:
        
        # Fetch column names
        for table_creation in statements:
            query = table_creation
            # print(query[:30]+'...')
            cur.execute(query)

    # with con.cursor() as cur2:    
        select = 'sel * from distinct_trips'
        
        cur.execute(select)
        rows = cur.fetchall()
        column_names = [desc[0] for desc in cur.description]

        # # query = statements
        # cur.execute(query)

        # column_names = [desc[0] for desc in cur.description]
        # rows = cur.fetchall()
        df = pd.DataFrame(rows)
        df.columns = column_names     

df

In [ ]:
host_dynamics_url = 'https://mgmhost.crm.dynamics.com/main.aspx?appid=df81b847-7ed4-4d00-bbc4-01b31d607d2a&pagetype=entitylist&etn=task&viewid=5a944623-fe03-ee11-8f6e-6045bd006149&viewType=4230&lid=1722024607660'
chrome_options = webdriver.ChromeOptions()

# chrome_options.add_experimental_option("prefs", {
#     "download.default_directory": download_dir,
# })

driver = webdriver.Chrome(options=chrome_options)
driver.get(host_dynamics_url)
print(host_dynamics_url)
# input_element = WebDriverWait(driver, 10).until(
#     EC.presence_of_element_located((By.CSS_SELECTOR, 'input.form-control.ltr_override.input.ext-input.text-box.ext-text-box'))
# )
# text_entry = driver.find_element(By.CSS_SELECTOR, 'input.form-control.ltr_override.input.ext-input.text-box.ext-text-box')
# text_entry.send_keys('cweber@mgmresorts.com')
# text_entry.send_keys(Keys.ENTER)

timeout = 1000
button = WebDriverWait(driver, timeout).until(
    EC.presence_of_element_located((By.ID, 'activitypointer|NoRelationship|HomePageGrid|Mscrm.HomepageGrid.activitypointer.ExportToExcel.Menu$button0'))
)

button.click()
print("Button clicked successfully.")

In [ ]:
length = 0
n = 0

host_dynamics_urls = ['https://mgmhost.crm.dynamics.com/main.aspx?appid=df81b847-7ed4-4d00-bbc4-01b31d607d2a&pagetype=entitylist&etn=task&viewid=5a944623-fe03-ee11-8f6e-6045bd006149&viewType=4230&lid=1722024607660'
                      , 'https://mgmhost.crm.dynamics.com/main.aspx?appid=df81b847-7ed4-4d00-bbc4-01b31d607d2a&pagetype=entitylist&etn=task&viewid=356ac88c-fc03-ee11-8f6e-6045bd006793&viewType=4230'
                      , 'https://mgmhost.crm.dynamics.com/main.aspx?appid=df81b847-7ed4-4d00-bbc4-01b31d607d2a&pagetype=entitylist&etn=task&viewid=feb7d7bc-fc03-ee11-8f6e-6045bd006793&viewType=4230']


# download_dir = 'C:/Users/Chweber/Host NBA CRM Stuff/CRM Files'

chrome_options = webdriver.ChromeOptions()

# chrome_options.add_experimental_option("prefs", {
#     "download.default_directory": download_dir,
# })

driver = webdriver.Chrome(options=chrome_options)
driver.get(host_dynamics_urls[0])
print(host_dynamics_urls[0])

files = os.listdir('C:/Users/Chweber/Downloads')
og_length = len(files)
new_length = og_length

while length != og_length + 3:
    while new_length < length+1:
        try:
            timeout = 1000  
            
            button = WebDriverWait(driver, timeout).until(
                EC.presence_of_element_located((By.ID, 'activitypointer|NoRelationship|HomePageGrid|Mscrm.HomepageGrid.activitypointer.ExportToExcel.Menu$button0'))
            )
            
            button.click()
            print("Button clicked successfully.")
            new_length = len(os.listdir('C:/Users/Chweber/Downloads'))
            time.sleep(120)
            
        except Exception as e:
            print(f"An error occurred: {e}")

    n += 1
    driver.get(host_dynamics_urls[n])
    
    updated_files = os.listdir('C:/Users/Chweber/Downloads')
    length = len(updated_files)

# driver.get('https://mgmhost.crm.dynamics.com/main.aspx?appid=df81b847-7ed4-4d00-bbc4-01b31d607d2a&pagetype=entitylist&etn=task&viewid=356ac88c-fc03-ee11-8f6e-6045bd006793&viewType=4230&lid=1722024426428')

# # time.sleep(35)
# WebDriverWait(driver, 1000).until(
#     EC.presence_of_element_located((By.ID, 'activityDueDateFilterComboBox-input'))
# )

button = driver.find_element(By.ID, 'activitypointer|NoRelationship|HomePageGrid|Mscrm.HomepageGrid.activitypointer.ExportToExcel.Menu$button0')
# all_elements = driver.find_elements(By.XPATH, '//*')

# # Print tag names of all elements
# for element in all_elements:
#     print(element.tag_name)

# driver.refresh()
# print(driver.page_source)